# Urban Green Space in Africa 

### Background
This notebook shows the [Sahel and West Africa Club's](https://www.oecd.org/swac/) (SWAC, OECD) approach to estimating urban green space within African urban agglomerations. Urban green spaces enhance a city's resilience to climate change and sustainability by mitigating extreme weather and gradual threats like drought and erosion. They reduce air pollution, sequester carbon, improve water quality, and support biodiversity and people's physical and mental health. A [SWAC analysis](https://www.oecd-ilibrary.org/development/boosting-african-cities-resilience-to-climate-change_3303cfb3-en) highlights these benefits for African cities and uncovers interesting continental trends.

The [ESA World Cover](https://esa-worldcover.org/) dataset offers information on land cover types, allowing for the calculation of land cover fractions such as **urban green space**, built-up areas, open land, and open water. Zonal statistics are used to derive frequency histograms of land cover classes within each agglomeration, which are then processed to calculate these indicators. **Urban green space** is calculated as the sum of the area of the classes *tree cover*, *shrubland*, and *grassland* divided by the total area of the urban agglomeration. See the [Mapping Territorial Transformations in Africa](https://www.mapping-africa-transformations.org/climate/) platform for more information and insights derived from this indicator.

#### Reference
Anderson, B., J.E. Patiño Quinchia, R. Prieto-Curiel (2022), “Boosting African cities’ resilience to climate change: The role of green spaces”, West African Papers, No. 37, OECD Publishing, Paris. https://doi.org/1787/3303cfb3-en 

### Description
This notebook is designed to derive urban green space metrics in Africa. The boundaries defining urban areas can be selected from the [AFRICAPOLIS](https://africapolis.org/) urban agglomerations feature collection or defined/uploaded by the user. The AFRICAPOLIS feature collection provides a comprehensive dataset of urban agglomerations across Africa, facilitating the analysis of land cover patterns. Alternatively, users can define their own areas of interest or upload custom datasets for analysis.

#### Workflow:
1. Select a location (urban agglomeration) for the analysis.
2. Calculate the frequency of land cover occurrence in each urban agglomeration of interest.
3. Calculate land cover fractions and aggregate into categories (e.g., green space, built-up, open land, open water).

### Load packages
Import Python packages that are used for the analysis.

In [1]:
import datacube
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm.auto import tqdm
from datacube.utils import geometry
from odc.dscache.tools import tiling
from shapely.geometry import Polygon, shape
from datacube.utils.geometry import Geometry

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.areaofinterest import define_area

### Connect to the datacube
Connect to the datacube so we can access DE AFrica data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
dc = datacube.Datacube(app="OECD_SWAC_Urban_green_space")

### Upload or select urban agglomeration(s)

The following cell sets the parameters, which define the area of interest to conduct the analysis over.
#### Select location
To define the area of interest, there are two methods available:

1. By specifying the latitude, longitude, and buffer, or separate latitude and longitude buffers, this method allows you to define an area of interest around a central point. You can input the central latitude, central longitude, and a buffer value in degrees to create a square area around the center point. For example, `lat = 10.338`, `lon = -1.055`, and `buffer = 0.1` will select an area with a radius of 0.1 square degrees around the point with coordinates `(10.338, -1.055)`. 
    
    Alternatively, you can provide separate buffer values for latitude and longitude for a rectangular area. For example, `lat = 10.338`, `lon = -1.055`, and `lat_buffer = 0.1` and`lon_buffer = 0.08` will select a rectangular area extending 0.1 degrees north and south, and 0.08 degrees east and west from the point `(10.338, -1.055)`.
    
    For reasonable loading times, set the buffer as `0.1` or lower.

2. By uploading a polygon as a `GeoJSON or Esri Shapefile`. If you choose this option, you will need to upload the geojson or ESRI shapefile into the Sandbox using Upload Files button <img align="top" src="../Supplementary_data/upload_files_icon.png"> in the top left corner of the Jupyter Notebook interface. ESRI shapefiles must be uploaded with all the related files `(.cpg, .dbf, .shp, .shx)`. Once uploaded, you can use the shapefile or geojson to define the area of interest. Remember to update the code to call the file you have uploaded.

To use one of these methods, you can uncomment the relevant line of code and comment out the other one. To comment out a line, add the `"#"` symbol before the code you want to comment out. By default, the first option which defines the location using latitude, longitude, and buffer is being used.

In [3]:
# Method 1: Specify the latitude, longitude, and buffer
# aoi = define_area(lat=-23.0475, lon=29.9165, buffer=0.05)

# Method 2: Use a polygon as a GeoJSON or Esri Shapefile. This can include multiple polygons.
aoi = define_area(vector_path='data/test.shp')

#Create a geopolygon and geodataframe of the area of interest
geopolygon_list = [Geometry(feature['geometry'], crs="EPSG:4326") for feature in aoi['features']]
agglomeration = gpd.GeoDataFrame(geometry=geopolygon_list, crs="EPSG:4326")
agglomeration['agglosID'] = range(1, len(agglomeration) + 1)

# Get the latitude and longitude range of the geopolygon
lat_range = (agglomeration.total_bounds[1], agglomeration.total_bounds[3])
lon_range = (agglomeration.total_bounds[0], agglomeration.total_bounds[2])
# display_map(x=lon_range, y=lat_range)

agglomeration.explore(tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
                attr='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}')

### Load AFRICAPOLIS urban agglomeration polygons
If you prefer to select an agglomeration from the AFRICAPOLIS urban agglomeration data, use the code below to make your selection. You can uncomment the lines of code by removing the `"#"` symbol before the code.

In [4]:
# afpolis = gpd.read_file("data/Africapolis2015.gpkg")
# afpolis.head()

#### Select urban agglomeration polygons for a country of interest

In [5]:
# print(afpolis['ISO3'].unique())

In [6]:
# afpolis_select = afpolis[afpolis['ISO3'] == 'ZAF']
# afpolis_select.explore(tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
#                 attr='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}')

##### Select urban agglomeration polygon of interest

In [7]:
# names_to_select = ['Thohoyandou']

# agglomeration = afpolis_select[afpolis_select['agglosName'].isin(names_to_select)]
# print(agglomeration)
# agglomeration.explore(tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
#                 attr='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}')

### Perform zonal statistics using ESA Worldcover (2020 or 2021)
The code in the cell below performs zonal statistics on an agglometation polygon or polygons contained in a GeoDataFrame. It iterates through each polygon, defining a query to load relevant ESA WorldCover data (for 2020 or 2021). The data for each polygon is rasterised, and the relevant area is masked to isolate the data within the polygon's boundary. Frequency histograms of land cover classes are then calculated from the raster data. These histograms are stored in a list along with the polygon's identifier. After processing all polygons, the results are converted to a DataFrame and exported to a CSV file.
> ***ESA_WorldCover_{year}_Zonal_Stats.csv:***
>   - This CSV file contains the raw histogram data for each polygon.
>   - Each row corresponds to a polygon, with the histogram data of land cover classes in a dictionary format.
>   - This file is useful for users who want to analyze the raw land cover class distribution data for each polygon.


In [8]:
# Define resolution, measurements, and classes
resolution = (-10, 10)
measurements = ['classification']
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
    
# Zonal stats
zs_results = []

# for idx, row in tqdm(afpolis.iterrows()):
# Iterate over each row in the GeoDataFrame with progress bar
for idx, row in tqdm(agglomeration.iterrows(), total=agglomeration.shape[0]):
    # Define query parameters based on the polygon's geometry
    geom = geometry.Geometry(geom=row['geometry'], crs="EPSG:4326") 

    # Create a reusable datacube query object
    query = {
        'time': ('2021'), # Select the year of interest for the ESA WorldCover product: either 2020 or 2021
        'geopolygon': geom,
        'resolution': resolution,
        'output_crs': 'EPSG:6933',  
        'measurements': measurements
    }

    try:
        # Load the ESA data for the current polygon
        ds_esa = dc.load(product=f'esa_worldcover_{query["time"]}', **query).squeeze()
        
        #Rasterise the area of interest polygon
        aoi_raster = xr_rasterize(gdf=gpd.GeoDataFrame(geometry=[row['geometry']], crs="EPSG:4326"),
                                  da=ds_esa,
                                  crs=ds_esa.crs)
        #Mask the dataset to the rasterised area of interest
        ds_esa = ds_esa.where(aoi_raster == 1)
        
        # Calculate frequency histogram of land cover classes
        lc_array = ds_esa.classification.values.flatten()
        lc_classes, lc_counts = np.unique(lc_array, return_counts=True)
        lc_frequency = dict(zip(lc_classes, lc_counts))
        
        # Remove last element (NaN) from dictionary
        lc_frequency.popitem()

        # Append results to zs_results
        zs_results.append({'agglosID': 	row.agglosID, 'histogram': lc_frequency})

    except Exception as e:
        # Handle any errors that occur during data loading or processing
        print(f"Error processing polygon {idx}: {e}")
    
# Once all polygons are processed, export zs_results to a CSV file or perform further analysis
# Convert results to DataFrame and save
zs_results_df = pd.DataFrame(zs_results)
zs_results_df.to_csv(f'ESA_WorldCover_{query["time"]}_Zonal_Stats.csv', index=False)
print(zs_results_df.head())


  0%|          | 0/4 [00:00<?, ?it/s]

   agglosID                                          histogram
0         1  {10.0: 30042, 20.0: 19063, 30.0: 117677, 40.0:...
1         2  {10.0: 1674882, 20.0: 215057, 30.0: 832174, 40...
2         3  {10.0: 8714, 20.0: 7369, 30.0: 13643, 40.0: 50...
3         4  {10.0: 49330, 20.0: 76170, 30.0: 62860, 40.0: ...


### Process histogram data into separate columns and calculate fractions
The code processes the ESA WorldCover zonal statistics results to calculate the fractions of land cover classes within each polygon. It converts histogram data into separate columns for each class, fills missing values with 0, and ensures the data is in integer format. Landcover fractions are calculated by dividing each class count by the total count and the results are ordered, selected, and exported to a CSV file, which contains the land cover class fractions for each polygon. 
>***ESA_WorldCover_{year}_OpenSpace.csv:***
>   - This CSV file contains the processed histogram data, where each land cover class is represented as a separate column.
>   - Each row corresponds to a polygon identified by the 'agglosID' column and includes the fraction of each land cover class relative to the total pixel count within that polygon.
>   - This file is particularly useful for users who need a detailed breakdown of land cover classes in a tabular format, enabling detailed examination and comparison of land cover distributions within the agglomeration polygon.


In [9]:
# Process histogram into separate columns
def process_histogram(histogram):
    return pd.Series(histogram)

histogram_df = zs_results_df['histogram'].apply(process_histogram).fillna(0).astype(int)
histogram_df = histogram_df.reindex(columns=classes, fill_value=0)

# Combine with original DataFrame
zs_ESA_LC = pd.concat([zs_results_df[['agglosID']], histogram_df], axis=1)

# Rename columns for clarity
rename_columns = {k: f'lc_{k}_count' for k in classes}
zs_ESA_LC.rename(columns=rename_columns, inplace=True)

# Calculate total counts and fractions
zs_ESA_LC['total_count'] = zs_ESA_LC.filter(like='lc_').sum(axis=1)
for k in classes:
    zs_ESA_LC[f'lc_{k}_fraction'] = zs_ESA_LC[f'lc_{k}_count'] / zs_ESA_LC['total_count']
    
# Explicitly order the fraction columns
fraction_columns_ordered = [
    "lc_50_fraction",  # Built-up
    "lc_10_fraction",  # Trees
    "lc_20_fraction",  # Shrubland
    "lc_30_fraction",  # Grassland
    "lc_40_fraction",  # Cropland
    "lc_60_fraction",  # Barren / sparse vegetation
    "lc_100_fraction", # Moss and lichen
    "lc_70_fraction",  # Snow and ice
    "lc_80_fraction",  # Open water
    "lc_90_fraction",  # Herbaceous wetland
    "lc_95_fraction"   # Mangroves
]    
zs_ESA_LC_selected = zs_ESA_LC[['agglosID'] + fraction_columns_ordered].fillna(0).round(5)
zs_ESA_LC_selected.to_csv(f'ESA_WorldCover_{query["time"]}_Landcover_Fractions.csv', index=False, float_format='%.5f')
zs_ESA_LC_selected.head()

,agglosID,lc_50_fraction,lc_10_fraction,lc_20_fraction,lc_30_fraction,lc_40_fraction,lc_60_fraction,lc_100_fraction,lc_70_fraction,lc_80_fraction,lc_90_fraction,lc_95_fraction
0,1,0.50901,0.05047,0.03203,0.19770,0.20527,0.00516,0.0,0.0,0.00037,0.00001,0.0
1,2,0.41494,0.26610,0.03417,0.13221,0.14664,0.00576,0.0,0.0,0.00015,0.00004,0.0
2,3,0.44170,0.06057,0.05122,0.09484,0.35063,0.00104,0.0,0.0,0.00000,0.00000,0.0
3,4,0.22659,0.13801,0.21310,0.17586,0.24247,0.00394,0.0,0.0,0.00001,0.00000,0.0


#### Adding land cover fractions to agglomeration polygons and exporting as shapefile
If desired, the calculated land cover fractions can be exported as a shapefile. The calculated land cover fractions from the ESA World Cover 2020 dataset are added to the agglomeration polygons so each polygon includes attributes representing the percentage of different land cover types within its boundaries. 
> ***ESA_WorldCover_{year}_OpenSpace.shp:***
>   - This shapefile retains the original geometries of the polygons and includes additional columns for the fraction of each land cover class
>   - It is particularly useful for spatial analysis and visualization in GIS software, offering both geometric and attribute data in a single file for analysis and visualisation of the distribution of land cover types across different polygons

In [10]:
agglomeration = agglomeration.merge(zs_ESA_LC_selected, on='agglosID', how='left')
agglomeration.to_file(f'ESA_WorldCover_{query["time"]}_Landcover_Fractions.shp')

/tmp/ipykernel_11624/779894502.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  agglomeration.to_file(f'ESA_WorldCover_{query["time"]}_Landcover_Fractions.shp')


### Calculate proxies for open space
The following code calculates proxies for open space based on the fractions of specific land cover classes.
> ***ESA_WorldCover_{year}_OpenSpace_Proxies.csv:***
>   - This CSV file contains calculated proxies for specific open space categories based on land cover fractions.
>   - This file provides a concise summary of key land cover characteristics for each polygon, useful for users interested in analyzing and comparing urban land cover distributions and open space within different polygons.


In [11]:
# Calculate specific fractions
# Built-up
zs_ESA_LC['ESA_LC_Built_up'] = zs_ESA_LC['lc_50_fraction']
# Calculate open land fraction
zs_ESA_LC['ESA_LC_Open_land'] = zs_ESA_LC[['lc_10_fraction', 'lc_20_fraction', 'lc_30_fraction', 
                                            'lc_40_fraction', 'lc_60_fraction', 'lc_100_fraction']].sum(axis=1)
# Calculate open water fraction
zs_ESA_LC['ESA_LC_Open_water'] = zs_ESA_LC[['lc_70_fraction', 'lc_80_fraction', 'lc_90_fraction', 'lc_95_fraction']].sum(axis=1)


# Calculate urban green space fraction
zs_ESA_LC['ESA_LC_Green_space'] = zs_ESA_LC[['lc_10_fraction', 'lc_20_fraction', 'lc_30_fraction']].sum(axis=1)

# Save final DataFrame
zs_ESA_LC[['agglosID', 'ESA_LC_Green_space', 'ESA_LC_Open_land', 'ESA_LC_Open_water', 'ESA_LC_Built_up']].to_csv(f'ESA_WorldCover_{query["time"]}_OpenSpace_Proxies.csv', index=False)
zs_ESA_LC[['agglosID', 'ESA_LC_Green_space', 'ESA_LC_Open_land', 'ESA_LC_Open_water', 'ESA_LC_Built_up']].head()


,agglosID,ESA_LC_Green_space,ESA_LC_Open_land,ESA_LC_Open_water,ESA_LC_Built_up
0,1,0.280191,0.490616,0.000378,0.509006
1,2,0.432479,0.584880,0.000184,0.414937
2,3,0.206636,0.558304,0.000000,0.441696
3,4,0.526977,0.773397,0.000014,0.226589


---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [12]:
print(datacube.__version__)

1.8.19


**Last Tested:**

In [13]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2024-11-01'